In [1]:
import os
os.chdir("../")

In [2]:
from pathlib import Path
from dataclasses import dataclass

In [3]:
@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file: Path
    target_column: str

In [4]:
from src.mlReg.constants import *
from src.mlReg.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = Config_File_Path,
            params_filepath = Params_File_Path,
            schema_filepath = Schema_File_Path
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.RandomForestRegressor
        schema = self.schema.TARGET_COLUMN 

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir= config.root_dir,
            test_data_path= config.test_data_path,
            model_path= config.model_path,
            all_params= params,
            metric_file= config.metric_file,
            target_column= schema.name
        )

        return model_evaluation_config

In [6]:
import os
import pandas as pd
import numpy as np
import joblib
from src.mlReg.utils.common import save_json
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score

In [7]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
    
    def evaluate_model(self, true, prediction):
        mae = np.round(mean_absolute_error(true, prediction), 2)
        mape = np.round(mean_absolute_percentage_error(true, prediction), 2)
        r2 = np.round(r2_score(true, prediction), 2)
        ad_r2 = np.round(1-((1 - r2)*(38715 - 1) / (38715 - 23 - 1)), 2)
        
        return mae, mape, r2, ad_r2
    
    def save_result(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        X_test = test_data.drop(columns=[self.config.target_column], axis=1)
        y_test = test_data[[self.config.target_column]]

        predict_price = model.predict(X_test)

        mae, mape, r2, adj_r2 = self.evaluate_model(y_test, predict_price)

        scores = {'Mean_Absolute_Error': mae, 
                  'Mean_Absolute_Percentage_Error': mape,
                  'R2_Score': r2,
                  'Adjusted_R2_Score': adj_r2}
        
        save_json(path= Path(self.config.metric_file), data= scores)

In [8]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config= model_evaluation_config)
    model_evaluation.save_result()
except Exception as e:
    raise e

[ [2024-03-11 17:50:07,770] : INFO : common : yaml file: config\config.yaml loaded successfully ]
[ [2024-03-11 17:50:07,776] : INFO : common : yaml file: params.yaml loaded successfully ]
[ [2024-03-11 17:50:07,788] : INFO : common : yaml file: schema.yaml loaded successfully ]
[ [2024-03-11 17:50:07,792] : INFO : common : created directory at: artifacts ]
[ [2024-03-11 17:50:07,796] : INFO : common : created directory at: artifacts/model_evaluation ]
[ [2024-03-11 17:50:08,355] : INFO : common : json file saved at: artifacts\model_evaluation\metrics.json ]
